In [8]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from autots import AutoTS
import pathlib
import json


In [9]:

gelman_data = pd.read_excel('../../data/Gelman_2020_DATA_Rockworks6-2020.xlsx',
                          sheet_name = 'TmInterval')

loc_data = pd.read_excel('../../data/Gelman_2020_DATA_Rockworks6-2020.xlsx',
                          sheet_name = 'Location')


In [10]:
loc_data.head(20)

,Bore,Enabled,Easting,Northing,Elevation,TotalDepth,CollarElevation,Comments,GEIcon,Color,...,spc_y,spc_unit,utm_datum,utm_zone,utm_x,utm_y,utm_unit,pj4_x,pj4_y,pj4_paramstr
0,1021 Westview Way,False,1.327472e+07,281015.800,1017.0000,268.0,-9999.00,NaN,1,0,...,0,1,22,12,0,0,1,0,0,NaN
1,1024 Westview Way,False,1.327422e+07,281040.300,1002.0800,-9999.0,-9999.00,NaN,1,0,...,0,1,22,12,0,0,1,0,0,NaN
2,1049 Westview Way,False,1.327472e+07,280891.300,1017.5300,280.0,-9999.00,NaN,1,0,...,0,1,22,12,0,0,1,0,0,NaN
3,105 Myrtle,False,1.326925e+07,286712.500,886.2000,65.0,884.00,Replacement water supply well.,1,0,...,0,1,22,12,0,0,1,0,0,NaN
4,1052 Westview Way,False,1.327430e+07,280863.700,1003.2100,276.0,-9999.00,NaN,1,0,...,0,1,22,12,0,0,1,0,0,NaN
5,1080 Westview Way,False,1.327433e+07,280728.800,1001.0800,276.0,-9999.00,NaN,1,0,...,0,1,22,12,0,0,1,0,0,NaN
6,110 Parkland Plaza,False,1.327283e+07,285907.000,936.1181,91.0,936.18,NaN,1,0,...,0,1,22,12,0,0,1,0,0,NaN
7,1105 Westview Way,False,1.327476e+07,280633.000,1015.7700,251.0,-9999.00,NaN,1,0,...,0,1,22,12,0,0,1,0,0,NaN
8,1108 Westview Way,False,1.327437e+07,280620.000,1000.8500,266.0,-9999.00,NaN,1,0,...,0,1,22,12,0,0,1,0,0,NaN
9,1130 Westview Way,False,1.327430e+07,280492.400,994.5400,282.0,-9999.00,NaN,1,0,...,0,1,22,12,0,0,1,0,0,NaN


In [5]:
from pyproj import CRS
from pyproj import Transformer
crs = CRS.from_proj4("+proj=lcc +lat_1=42.1 +lat_2=43.66666666666666 +lat_0=41.5 +lon_0=-84.36666666666666 +x_0=4000000 +y_0=0 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 no_defs")

crs_4326 = CRS.from_epsg(4326)

transformer = Transformer.from_crs(crs, crs_4326)

x1,y1  = 13274717.9 , 281015.8

transformer.transform(x1,y1)


(42.269731983117495, -83.8073438786552)

In [13]:
loc_copy = loc_data.copy()
loc_copy.loc[:, 'lat'] = transformer.transform(loc_copy["Easting"],
                                               loc_copy["Northing"])[0]
loc_copy.loc[:, 'lon'] = transformer.transform(loc_copy["Easting"],
                                               loc_copy["Northing"])[1]
loc_copy.head(5)

,Bore,Enabled,Easting,Northing,Elevation,TotalDepth,CollarElevation,Comments,GEIcon,Color,...,utm_datum,utm_zone,utm_x,utm_y,utm_unit,pj4_x,pj4_y,pj4_paramstr,lat,lon
0,1021 Westview Way,False,13274717.9,281015.8,1017.00,268.0,-9999.0,NaN,1,0,...,22,12,0,0,1,0,0,NaN,42.269732,-83.807344
1,1024 Westview Way,False,13274221.6,281040.3,1002.08,-9999.0,-9999.0,NaN,1,0,...,22,12,0,0,1,0,0,NaN,42.269808,-83.809177
2,1049 Westview Way,False,13274722.1,280891.3,1017.53,280.0,-9999.0,NaN,1,0,...,22,12,0,0,1,0,0,NaN,42.269390,-83.807331
3,105 Myrtle,False,13269250.0,286712.5,886.20,65.0,884.0,Replacement water supply well.,1,0,...,22,12,0,0,1,0,0,NaN,42.285462,-83.827411
4,1052 Westview Way,False,13274301.3,280863.7,1003.21,276.0,-9999.0,NaN,1,0,...,22,12,0,0,1,0,0,NaN,42.269322,-83.808887


In [29]:
result = pd.merge(gelman_data, loc_copy, on=["Bore", "Bore"])

remove_invalid = result.loc[result['Easting'] != -9999].copy()

In [31]:
remove_invalid.to_excel("../../result(new)/joined_well_data.xlsx")
